In [ ]:
import numpy as np
from scipy.ndimage import convolve1d
from matplotlib import pyplot as plt


## Noise Augmentation
#Function for creating a noise vector, must be a 5000x1 array of 0's and 1's. Try to keep noise below xx

def add_gaussian_noise_via_convolution_1d(data, sigma_range, E_int, max_chunk_size):
	"""
	Add Gaussian noise through convolution using a Gaussian kernel to chunks of data with randomized sizes.

	Parameters:
		data (numpy.ndarray): Input data array (1D, e.g., 5000x1).
		sigma_range (tuple): Range of sigma values for Gaussian smoothing (min, max).
		kernel_size (int): Size of the Gaussian kernel.
		max_chunk_size (int): The maximum chunk size for low sigma values.

	Returns:
		numpy.ndarray: Data with added Gaussian noise.
	"""
	assert len(data.shape) == 1, "Input data must be a 1D array."

	kernel_size = max(1, int(np.round(1.5 / E_int)))
	modified_data = data.copy()  # Create a copy to modify
	total_length = len(data)  # Total data length
	idx = 0  # Start index for modification

	while idx < total_length:
		# Randomly select a sigma value from the range
		sigma = np.random.uniform(sigma_range[0], sigma_range[1])

		# Scale chunk size inversely to sigma
		chunk_size = int(max_chunk_size * (1 - (sigma / sigma_range[1])))
		chunk_size = max(1, chunk_size)  # Ensure at least one element is modified

		# Determine end index for this chunk
		end_idx = min(idx + chunk_size, total_length)

		# Extract the current chunk
		chunk = modified_data[idx:end_idx]

		# Generate a 1D Gaussian kernel
		ax = np.linspace(-(kernel_size - 1) / 2., (kernel_size - 1) / 2., kernel_size)
		kernel = np.exp(-0.5 * (ax / sigma) ** 2)
		kernel /= np.sum(kernel)  # Normalize the kernel

		# Convolve the kernel with the 1D chunk
		smoothed_chunk = convolve1d(chunk, kernel, mode='reflect')

		# Add random Gaussian noise
		noise = np.random.normal(loc=0, scale=sigma, size=chunk.shape)
		noisy_chunk = smoothed_chunk + noise

		# Update the modified data with the noisy chunk
		modified_data[idx:end_idx] = noisy_chunk

		# Move to the next chunk
		idx = end_idx

	return modified_data

def random_broadening(x: np.array, y: np.array, random_seed=42, sigma_min=10, sigma_max=100, kernel_size=16):
	broadened_y = np.zeros_like(y)
	half_kernel_size = kernel_size // 2
	x_kernel = np.linspace(-5, 5, kernel_size)

	np.random.seed(random_seed)  # random seed for reproducibility
	random_sigmas = np.random.uniform(sigma_min, sigma_max, size=len(x))  # random sigmas for each point

	for i in range(len(x)):
		# initialize kernel
		sigma = random_sigmas[i]
		kernel = np.exp(-x_kernel ** 2 / (2 * sigma ** 2))
		kernel /= kernel.sum()  # normalize kernel

		start = max(0, i - half_kernel_size)
		end = min(len(x), i + half_kernel_size + 1)

		kernel_start = max(0, half_kernel_size - i)
		kernel_end = kernel_start + (end - start)

		if kernel_end > len(kernel):
			kernel_end = len(kernel)
			end = start + (kernel_end - kernel_start)

		broadened_y[start:end] += y[i] * kernel[kernel_start:kernel_end]

	return broadened_y

## Shift Augmentation
def shift_spectra(energy_interval, energy_shift, intensity):
	random_shift = np.random.uniform(-energy_shift, energy_shift, size=(intensity.shape[0], 1))
	shifted_energy = random_shift / energy_interval
	shifted_energy = shifted_energy.astype(int)
	intensity = intensity.copy()
	for idx, inten in enumerate(intensity):
		intensity[idx] = np.roll(inten, shifted_energy[idx])

	return intensity


## Total Augmentation pipeline

#load dataset
path = '../dataset'
x_train = np.load(f'{path}/x_train_use.npy')
y_train = np.load(f'{path}/y_train_use.npy')


# Add noise to the training data

def add_noise_total(x_train, energy_interval, y_train):
	augment_strength = [0.01, 0.05, 0.1]
	x_train_augment = x_train.copy()
	x_train_augmented = np.zeros((len(augment_strength), x_train.shape[0], x_train.shape[1]))
	for idx1, x_train_sample in enumerate(x_train_augment):
		for idx, i in enumerate(augment_strength):
			sigma_min_max = (i * np.max(x_train_sample), (i + 0.05) * np.max(x_train_sample))
			x_train_augmented[idx][idx1] = add_gaussian_noise_via_convolution_1d(x_train_sample, sigma_min_max, 0.3, 300)

	y_train_augmented = y_train.copy()
	y_train_augmented = y_train_augmented.reshape((1, y_train_augmented.shape[0], y_train_augmented.shape[1]))
	y_train_augmented = np.tile(y_train_augmented, (x_train_augmented.shape[0], 1, 1))
	return x_train_augmented, y_train_augmented


def add_shift_total(x_train, energy_interval, y_train):
	energy_shift = 4
	x_train_augment = x_train.copy()
	x_train_augmented = np.zeros((1, x_train.shape[0], x_train.shape[1]))
	for idx1, x_train_sample in enumerate(x_train_augment):
		x_train_augmented[0][idx1] = shift_spectra(0.3, energy_shift, x_train_sample)

	y_train_augmented = y_train.copy()
	y_train_augmented = y_train_augmented.reshape((1, y_train_augmented.shape[0], y_train_augmented.shape[1]))
	y_train_augmented = np.tile(y_train_augmented, (x_train_augmented.shape[0], 1, 1))
	return x_train_augmented, y_train_augmented


def add_broadening_total(x_train, y_train):
	x_train_augment = x_train.copy()
	augment_strength = [10, 20, 30]
	x_train_augmented = np.zeros((len(augment_strength), x_train.shape[0], x_train.shape[1]))
	for idx1, x_train_sample in enumerate(x_train_augment):
		for idx, i in enumerate(augment_strength):
			x = np.arange(len(x_train_sample))
			x_train_augmented[idx][idx1] = random_broadening(x, x_train_sample, random_seed=42, sigma_min=10, sigma_max=100,
															 kernel_size=i)

	y_train_augmented = y_train.copy()
	y_train_augmented = y_train_augmented.reshape((1, y_train_augmented.shape[0], y_train_augmented.shape[1]))
	y_train_augmented = np.tile(y_train_augmented, (x_train_augmented.shape[0], 1, 1))
	return x_train_augmented, y_train_augmented


def add_augment_total(x_train, energy_interval, y_train):
	noise_augment = add_noise_total(x_train, energy_interval, y_train)
	shift_augment = add_shift_total(x_train, energy_interval, y_train)
	broaden_augment = add_broadening_total(x_train, y_train)
	total_augment_x = np.concatenate((noise_augment[0], shift_augment[0], broaden_augment[0]), axis=0)
	total_augment_y = np.concatenate((noise_augment[1], shift_augment[1], broaden_augment[1]), axis=0)
	return total_augment_x, total_augment_y


energy_interval = 'use'
x_train_augmented, y_train_augmented = add_augment_total(x_train, energy_interval, y_train)
x_train_augmented_res = x_train_augmented.reshape(
	(x_train_augmented.shape[0] * x_train_augmented.shape[1], x_train_augmented.shape[2]))
y_train_augmented_res = y_train_augmented.reshape(
	(y_train_augmented.shape[0] * y_train_augmented.shape[1], y_train_augmented.shape[2]))
x_train_total = np.concatenate((x_train, x_train_augmented_res), axis=0)
y_train_total = np.concatenate((y_train, y_train_augmented_res), axis=0)
np.save(f'{path}/x_train_augmented_{energy_interval}.npy', x_train_total)
np.save(f'{path}/y_train_augmented_{energy_interval}.npy', y_train_total)